In [ ]:
!pip install spacy
!python -m spacy download en_core_web_lg

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import random
import pandas as pd
from datetime import timedelta
import spacy

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
truth_seeker = '/content/drive/MyDrive/fakeBN/data/Truth_Seeker_Model_Dataset.csv'
truth_seeker_ml = '/content/drive/MyDrive/fakeBN/data/Features_For_Traditional_ML_Techniques.csv'
truth_seeker_timestamp = '/content/drive/MyDrive/fakeBN/data/Truth_Seeker_Model_Dataset_With_TimeStamps.csv'
significant_events =  '/content/drive/MyDrive/fakeBN/data/on_this_day_events_2009_2022.csv'

data_ts = pd.read_csv(truth_seeker)
data_ml = pd.read_csv(truth_seeker_ml)
data_timestamp = pd.read_csv(truth_seeker_timestamp)
data_significant_events = pd.read_csv(significant_events)

In [ ]:
data_ml_merged = data_ml.merge(data_timestamp[['tweet', 'timestamp']], how='left', on='tweet')
data_ml_merged = pd.concat([data_ml_merged, data_ml.drop('tweet', axis=1)], axis=1)
data_ml_merged.dropna(subset=['timestamp'], inplace=True)
data_ml_merged['timestamp'] = pd.to_datetime(data_ml_merged['timestamp'])
data_significant_events['date'] = pd.to_datetime(data_significant_events['date']).dt.date

In [ ]:
final_data = data_ml_merged.copy()

nlp_lg = spacy.load("en_core_web_lg")

# Function to check if a tweet is similar to significant events
def is_similar(row):
    start_date = (row['timestamp'] - pd.DateOffset(months=1)).date()
    end_date = row['timestamp'].date()
    filtered_events = data_significant_events[
        (data_significant_events['date'] >= start_date) &
        (data_significant_events['date'] <= end_date)
    ]
    tweet_doc = nlp_lg(row['tweet'])
    for event in filtered_events['event']:
        event_doc = nlp_lg(event)
        if tweet_doc.similarity(event_doc) >= 0.8:
            return 1
    return 0


final_data['matches_significant_event'] = final_data.apply(is_similar, axis=1)

final_data['timestamp'] = pd.to_datetime(final_data['timestamp'])


final_data['day'] = final_data['timestamp'].dt.dayofweek
final_data['month'] = final_data['timestamp'].dt.month
final_data['hour'] = final_data['timestamp'].dt.hour
final_data['minute'] = final_data['timestamp'].dt.minute
final_data['quarter'] = final_data['timestamp'].dt.quarter
final_data['isWeekend'] = final_data['timestamp'].dt.dayofweek.apply(lambda x: 1 if x >= 5 else 0)
final_data['day'] = final_data['timestamp'].dt.dayofweek
final_data['month'] = final_data['timestamp'].dt.month
final_data['hour'] = final_data['timestamp'].dt.hour
final_data['minute'] = final_data['timestamp'].dt.minute
final_data['quarter'] = final_data['timestamp'].dt.quarter
final_data['is_weekend'] = final_data['timestamp'].dt.dayofweek.apply(lambda x: 1 if x >= 5 else 0)

final_data.to_csv('/content/drive/MyDrive/fakeBN/data/final_data.csv', index=False)